### Boiler Plate Imports

In [72]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from typing import TypedDict, List, Literal, Optional, Tuple
from langgraph.graph import StateGraph, END

import sqlite3
from dotenv import load_dotenv
load_dotenv(override=True)

True

### Module: Define the State (Shared Memory)

In [73]:
# Define Graph State
class State(TypedDict):
    iteration: int
    topic: str
    post_content: Optional[str]
    post_content_revise: Optional[str]
    critique_feedback: str
    verdict: Optional[Literal["good", "rework"]]

### Module: Define Prompt Function

In [74]:
# Function to return prompt to generate LinkedIn post
def get_generate_post_prompt() -> str:
    
    # Define System Content
    systemContent = """
    You are an expert LinkedIn content strategist specializing in creating highly engaging posts that drive meaningful professional conversations.
    Your task is to generate a LinkedIn post about {topic} while incorporating any previous feedback provided in {critique-feedback}.

    Key Objectives:
    Generate posts that spark professional discussions and encourage meaningful engagement
    Maintain authenticity and avoid overly promotional language
    Incorporate storytelling elements when relevant
    Follow LinkedIn best practices for content structure and formatting
    Adapt and improve based on provided critique feedback

    Context Understanding:
    Topic: {topic}
    Previous Feedback: {critique-feedback}
    [Note: For initial generation, feedback will be empty. For subsequent iterations, incorporate the feedback to improve the post.]
    """

    # List of Messages for the LLM
    messages = [("system", systemContent), ("human", "{topic} {critique-feedback}")]

    # Return the Prompt
    prompt = ChatPromptTemplate.from_messages(messages)
    return prompt

# Function to return prompt to critique the LinkedIn post
def get_critique_post_prompt() -> str:
    
    # Define System Content
    systemContent = """
    You are an expert LinkedIn content analyst specializing in evaluating professional posts for maximum engagement and impact.
    Your task is to analyze the provided LinkedIn post and deliver two outputs: detailed critique feedback and a binary evaluation (good/rework).
    
    Input: {post}
    
    Analysis Framework:

    Engagement Potential Assessment:
    Hook strength and immediate attention grab
    Story arc and narrative flow
    Call-to-action effectiveness
    Discussion potential
    Emotional resonance

    Technical Elements Review:
    Length optimization (800-1300 characters ideal)
    Format and readability
    Emoji usage (appropriateness and quantity)
    Hashtag implementation
    Line break utilization

    Content Quality Evaluation:
    Value proposition clarity
    Professional tone consistency
    Authenticity markers
    Industry relevance
    Audience alignment

    Output Structure:
    Feedback: Detailed feedback about the post
    Verdict: Only output either the word 'good' or 'rework' in lowercase. Remember just a single word and nothing else

    Please note do not give the verdict as good when you get the post for the first time
    """

    # List of Messages for the LLM
    messages = [("system", systemContent), ("human", "{post}")]

    # Return the Prompt
    prompt = ChatPromptTemplate.from_messages(messages)
    return prompt

### Module: Define Nodes

In [75]:
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Node 1: Generate Post
def generate_node_function(state: State) -> State:

    # Get the Prompt
    prompt = get_generate_post_prompt()

    # Define the chain
    generate_post_chain = prompt | llm

    # Invoke the Chain        
    response = generate_post_chain.invoke({"topic": state["topic"], "critique-feedback": state["critique_feedback"]})

    if state["critique_feedback"] == "":
        # Update and Return the State
        return {
            **state,
            "post_content": response.content
        }
    else:
        # Update and Return the State
        return {
            **state,
            "post_content_revise": response.content
        }

# Node 2: Critique Post
def critique_post_node_function (state: State) -> State:

    # Get the Prompt
    prompt = get_critique_post_prompt()
    
    # Define the Chain
    critique_post_chain = prompt | llm

    # Invoke the Chain
    if state["post_content_revise"] == "":
        response = critique_post_chain.invoke({"post": state["post_content"]})
    else:
        response = critique_post_chain.invoke({"post": state["post_content_revise"]})

    # Get the feedback and Verdict from the response content
    parts = response.content.split('Verdict:')
        
    # Extract feedback (remove 'Feedback: ' prefix and strip whitespace)
    feedback = parts[0].replace('Feedback:', '', 1).strip()

    # Extract verdict (strip whitespace)
    verdict = parts[1].strip()

    # Update Iteration
    new_iteration = state.get("iteration") + 1

    # Update and Return the State
    return {
        **state,
        "critique_feedback": feedback,
        "verdict": verdict,
        "iteration": new_iteration
    }

### Module: Continue Condition

In [76]:
def continue_condition(state:State) -> str:

    # Get for verdict
    if state["verdict"].lower() == "rework" and state["iteration"] < 6:
        return "continue"
    else:
        return "stop"


### Module: Define Workflow

In [77]:
# Define workflow
workflow = StateGraph(State)

# Add nodes to the workflow
workflow.add_node("generate_post_node", generate_node_function)
workflow.add_node("critique_post_node", critique_post_node_function)

# Add a directed edge
workflow.add_edge("generate_post_node","critique_post_node")

# Add a conditional edge
workflow.add_conditional_edges(
    "critique_post_node",
    lambda x: continue_condition(x),
    {
        "continue": "generate_post_node",
        "stop": END
    }

)

# Set entry point
workflow.set_entry_point("generate_post_node")

### Module: Compile and Execute Workflow

In [78]:
# Initialize
app = workflow.compile()

# Get the user input about the topic of post
user_input = input("On which topic you want me to generate a LinkedIn post: ")

# Initialize state
state = {'iteration': 0, 'topic': user_input, 'critique_feedback': "", 'post_content_revise': ""}

# Process through workflow
result = app.invoke(state)

### Module: Print the Final Result

In [ ]:
print(f"Original LinkedIn post about {user_input}:")

print(result.get("post_content"))

print(f"\n Revise LinkedIn post about {user_input}:")

print(result.get("post_content_revise"))

In [79]:
result

{'iteration': 1,
 'topic': 'LangGraph',
 'post_content': "In the ever-evolving world of technology, it's crucial to stay ahead of the curve. Today, I'd like to introduce you to a tool that's been making waves in the language processing field - LangGraph.\n\nLangGraph is not just another tool; it's a revolution in the way we understand and process language data. It's a graph-based language model that leverages the power of machine learning to provide deeper insights into language patterns and structures.\n\nImagine being able to visualize the intricate web of connections between words, phrases, and sentences. With LangGraph, this is not just a dream but a reality. It's like having a bird's eye view of a city, where each word is a building, each sentence a street, and the entire language a sprawling metropolis.\n\nBut why does this matter to you? Well, if you're in the field of data analysis, AI, machine learning, or even content creation, LangGraph can be a game-changer. It can help you